# 📚 5강: 시험 대비 패키지

> 1페이지 요약, 빈칸 워크시트, 단어카드 CSV, 3주 스케줄을 한 번에 생성합니다.

## ⚠️ 시작 전 필수! 노트북 복사하기

이 노트북은 **읽기 전용**입니다. 실습하려면 반드시 복사본을 만드세요!

```
📌 상단 메뉴 → 파일 → 드라이브에 사본 저장
```

복사본이 내 Google Drive에 저장되면, 그 파일에서 코드를 수정하고 실행할 수 있습니다.

In [ ]:
# ⚙️ 환경 설정
!pip install -q -U google-genai

from google import genai
from google.colab import userdata
import getpass

try:
    api_key = userdata.get('GEMINI_API_KEY')
    if isinstance(api_key, dict):
        api_key = str(list(api_key.values())[0])
    API_KEY = str(api_key).strip()
except Exception:
    API_KEY = getpass.getpass('🔑 Gemini API Key를 입력하세요: ')

client = genai.Client(api_key=API_KEY)
MODEL = 'gemini-2.5-flash'
print('✅ 준비 완료!')

In [ ]:
# ✏️ 시험 정보
academy_name = "달빛영어"
exam_info = """대상: 초5 B반
시험 범위: Unit 5~8
핵심 문법: 현재진행형, 비교급, There is/are
핵심 어휘: 약 40개 (교재 기반)
시험 날짜: 2주 후"""

## 🔬 실습 1: 1페이지 요약 노트

In [ ]:
prompt1 = f"""
너는 영어교육 전문가이자 학습자료 디자이너야.

아래 시험 범위의 1페이지 요약 노트를 만들어줘.

{exam_info}

구성:
📌 핵심 문법 정리 (규칙 + 예문 2개씩)
📌 필수 어휘 20개 (영어 | 뜻 | 예문)
📌 자주 틀리는 포인트 TOP 5
📌 시험 팁 3가지

A4 한 장에 들어갈 분량. 시각적으로 정리.
"""

response = client.models.generate_content(model=MODEL, contents=prompt1)
summary = response.text
print('📋 1페이지 요약 노트')
print('='*50)
print(summary)

## 🔬 실습 2: 빈칸 채우기 워크시트

In [ ]:
prompt2 = f"""
너는 영어 시험 출제 전문가야.

아래 범위에서 빈칸 채우기 워크시트 15문항을 만들어줘.

{exam_info}

형식:
1. The boy is _______ (swim) in the pool.
   → 정답: swimming

난이도: 쉬움 5개 / 보통 5개 / 어려움 5개
각 문항에 정답도 함께 표시.
"""

response = client.models.generate_content(model=MODEL, contents=prompt2)
worksheet = response.text
print('✏️ 빈칸 채우기 워크시트')
print('='*50)
print(worksheet)

## 🔬 실습 3: 단어카드 CSV 생성

In [ ]:
prompt3 = f"""
너는 영어 어휘 교육 전문가야.

아래 시험 범위의 핵심 어휘 30개를 CSV 형식으로 만들어줘.

{exam_info}

CSV 형식 (헤더 포함):
English,Korean,Example,Difficulty
tall,키가 큰,He is tall.,easy

Difficulty: easy / medium / hard
30개 정확히. 헤더 포함.
"""

response = client.models.generate_content(model=MODEL, contents=prompt3)
csv_content = response.text

csv_clean = csv_content.replace('```csv', '').replace('```', '').strip()

with open('vocabulary_cards.csv', 'w', encoding='utf-8') as f:
    f.write(csv_clean)

import pandas as pd
try:
    vocab_df = pd.read_csv('vocabulary_cards.csv')
    print(f'📇 단어카드 {len(vocab_df)}개 생성!')
    print(vocab_df.head(10).to_string(index=False))
except Exception as e:
    print(f'CSV 파싱 참고: {e}')
    print(csv_clean[:500])

## 🔬 실습 4: 3주 시험 대비 스케줄

In [ ]:
prompt4 = f"""
너는 학습 플래너 전문가야.

시험까지 3주 남은 학생의 시험 대비 스케줄을 만들어줘.

{exam_info}

| 주차 | 목표 | 월 | 화 | 수 | 목 | 금 | 주말 |
|---|---|---|---|---|---|---|---|
| 1주차 | 개념 정리 | ... | ... | ... | ... | ... | ... |
| 2주차 | 문제 풀이 | ... | ... | ... | ... | ... | ... |
| 3주차 | 최종 점검 | ... | ... | ... | ... | ... | ... |

하루 30분 기준. 주말은 복습 + 모의시험.
"""

response = client.models.generate_content(model=MODEL, contents=prompt4)
schedule = response.text
print('📅 3주 시험 대비 스케줄')
print('='*50)
print(schedule)

In [ ]:
# 💾 저장
output = f"요약:\n{summary}\n\n워크시트:\n{worksheet}\n\n스케줄:\n{schedule}"
with open('시험대비_패키지.txt', 'w', encoding='utf-8') as f:
    f.write(output)
from google.colab import files
files.download('시험대비_패키지.txt')
files.download('vocabulary_cards.csv')
print('✅ 다운로드!')

## 🏆 5강 완료!
👉 [6강: 성적 분석 & 위험 감지](https://colab.research.google.com/github/Reasonofmoon/cedu-1/blob/main/notebooks/06_risk_radar.ipynb)